<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# Enhancing Data Science Outcomes With Efficient Workflow #

## 04 - Embeddings ##
In this lab, you will use high-performance computing to create machine learning solutions. This lab covers the model development portion of the data science workflow. A good machine learning solution excels that both accuracy and inference performance. 

<p><img src='images/pipeline_overview_2.png' width=1080></p>

**Table of Contents**
<br>
This notebook covers the below sections: 
1. [Entity Embedding](#s4-1)
2. [Training the Embeddings](#s4-2)
    * [Preparing the Data - Normalization](#s4-2.1)
    * [Model Building](#s4-2.2)
    * [Being Training](#s4-2.3)
3. [Visualizing the Embeddings](#s4-3)
4. [Conclusion](#s4-4)

<a name='s4-1'></a>
## Entity Embeddings ##
[Entity Embeddings](https://arxiv.org/pdf/1604.06737.pdf) are very similar to word embeddings used in NLP. They are a way to represent categorical features in a defined latent space. In the latent space, categories that are semantically similar have similar vectors. Embeddings can be trained to assign a learnable feature vector to each category. Using embeddings, each categorical value is mapped to its own associated vector representation that is more informative than a single point value. Even though embeddings require a large amount of data and computational resources to train, they have proven to be a great alternative encoding method to consider. Once trained, embeddings can boost the performance of downstream machine learning tasks when used as the input features. Users can combine the power of deep learning with traditional machine learning on tabular data. 

<p><img src='images/embedding.png' width=720></p>

Reasons for using embeddings include: 
* It is much more efficient than the one-hot approach for encoding when cardinality if high
* Allows rich relationships and complexities between categories to be captured
* Reduce memory usage and speed up downstream machine learning model training
* Once trained, the same embedding can be used for various use cases
* Can be used to visualize categorical data and for data clustering, since the embedding space quantifies semantic similarity as distance between the categories in the latent space
* Mitigates the need to perform cumbersome manual feature engineering, which requires extensive domain knowledge

<p><img src='images/tip.png' width=720></p>

Below are some tips about embeddings: 
* Requires training with large amounts of data, making it inappropriate for unseen data such as when new categories are added
* Can overfit
* Difficult to interpret

<a name='s4-2'></a>
## Training the Embeddings ##
Embeddings aim to represent each entity as a numeric vector such that products in similar context have similar vectors. Mathematically, similar entities will have a large dot product whereas every entity when one-hot encoded has a zero dot product with every other entity. This is because all one-hot vectors are orthogonal. 

We will use [PyTorch](https://pytorch.org/) to train a simple fully-connected neural network. A surrogate problem is setup for the purpose of finding the embedding vectors. Neural networks have difficultly with sparse categorical features. Traditionally, embeddings are a way to reduce those features to increase model performance. 

Technically, the idea of an embedding layer is very similar to a dense or linear layer (without bias) in the neural network. When training an embedding this way, users will one-hot encode the categorical data so each record becomes a vector with C features, where C is the cardinality. We then perform matrix vector multiplication on the input vector and the weights before feeding the next layer. This is inefficient when the number of input features is large and sparse, as is the case for categorical features from a tabular dataset. 

A better and more efficient approach would be to train a `torch.nn.Embedding` layer, which can be treated as a "lookup" table with the label-encoded category id as the index. By using choosing this, we avoid one-hot encoding and the matrix vector multiplication. 

<p><img src='images/surrogate_problem.png' width=720></p>

<p><img src='images/tip.png' width=720></p>

Embeddings will naturally be affected by how the surrogate problem is defined. 

In [1]:
# import dependencies
from tqdm import tqdm
import cudf
import cuml
import dask_cudf

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as torch_optim
from torch.utils.data import Dataset, DataLoader

In [2]:
# set device cuda to use GPU
device=torch.device('cuda')

In [3]:
# define features and label
cols=['brand', 'cat_0', 'cat_1', 'cat_2', 'price', 'target']
cat_cols=['brand', 'cat_0', 'cat_1', 'cat_2']
label='target'

feature_cols=[col for col in cols if col != label]
cont_cols=[col for col in feature_cols if col not in cat_cols] # ['price']

In [4]:
# read data
parquet_dir='processed_parquet'

ddf=dask_cudf.read_parquet(parquet_dir, columns=cols)
gdf=ddf.compute()

print(f'Total of {len(gdf)} records.')

Total of 2461697 records.


<p><img src='images/tip.png' width=720></p>

Even though we intend to keep all the data in one GPU, we still recommend loading data with `Dask-cuDF`. 

In [5]:
gdf.head()

,brand,cat_0,cat_1,cat_2,price,target
0,1,6,5,2,100.229996,1
1,2,1,1,1,871.839966,1
2,2,1,1,1,872.090027,1
3,2,6,5,2,306.690002,1
4,13,2,3,24,334.349976,1


In [6]:
# the embedding vectors will start with 0 so we decrease the categorical values by 1 to match
gdf[cat_cols]=gdf[cat_cols]-1

n_uniques=gdf.nunique()

# use higher of 4th root of nunique and 3 for vector dimension
embedding_sizes={col: (n_uniques[col], max(3, int(n_uniques[col]**0.25))) for col in cat_cols}
embedding_sizes

{'brand': (3303, 7), 'cat_0': (14, 3), 'cat_1': (61, 3), 'cat_2': (90, 3)}

<p><img src='images/tip.png' width=720></p>

The size of embeddings can become very large. For example, large embeddings are usually needed for users and items for large platforms. 

<a name='s4-2.1'></a>
### Preparing the Data - Normalization ###
**Normalization** is required to enable neural networks to leverage numerical features. Tree-based models do not require normalization as they define the split independent of the scale of a feature. Without normalization, neural networks are difficult to train. The reason is that different numerical features have different scales. When we combine the features in a hidden layer, the different scales make it more difficult to extract patterns from it. 

<p><img src='images/tip.png' width=720></p>

We will also implement a `torch.nn.BatchNorm1d`[[doc]](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html) layer to mitigate the exploding gradient problem. 

In [7]:
# normalize data
gdf['price']=cuml.preprocessing.StandardScaler().fit_transform(gdf[['price']])

<a name='s4-2.2'></a>
### Model Building ###
We construct a model with several layers. The embeddings will be the same dimension as num_unique x vector_size. The embeddings will be concatenated, along with the continous variable(s), before they are fed into the next layer. 

In [8]:
# define neural network with embedding layers
class ProductPurchaseModel(nn.Module):
    def __init__(self, embedding_sizes, n_cont):
        super().__init__()
        # make an embedding for each categorical feature
        # The `nn.Embedding` layer can be thought of as a lookup table where the key is 
        # the category index and the value is the corresponding embedding vector
        self.embeddings=nn.ModuleList([nn.Embedding(n_categories, size) for n_categories, size in embedding_sizes.values()])
        
        # n_emb is the length of all embeddings combined
        n_emb=sum(e.embedding_dim for e in self.embeddings)
        
        self.n_emb=n_emb
        self.n_cont=n_cont
        self.emb_drop = nn.Dropout(0.6)
        
        # apply dropout, batch norm and linear layers
        self.bn1=nn.BatchNorm1d(self.n_cont)
        self.lin1=nn.Linear(self.n_emb + self.n_cont, 200)
        self.drop1=nn.Dropout(0.3)
        self.bn2=nn.BatchNorm1d(200)
        self.drop2=nn.Dropout(0.3)
        self.lin2=nn.Linear(200, 70)
        self.bn3=nn.BatchNorm1d(70)
        self.lin3=nn.Linear(70, 2)

    def forward(self, X_cat, X_cont):
        # map each categorical feature to the embedding vector on its corresponding embedding layer
        x_1=[embedding(X_cat[:, idx]) for idx, embedding in enumerate(self.embeddings)]
        
        # concatenate all categorical embedding vectors together
        x_1=torch.cat(x_1, 1)
        
        # apply random drop out, normalization, and activation
        x_1=self.emb_drop(x_1)
        x_2=self.bn1(X_cont)
        
        # concatenate categorical embeddings to input layer from continuous variable(s)
        x_1=torch.cat([x_1, x_2], 1)
        
        # apply random drop out, normalization, and activation
        x_1=F.relu(self.lin1(x_1))
        x_1=self.drop1(x_1)
        x_1=self.bn2(x_1)
        x_1=F.relu(self.lin2(x_1))
        x_1=self.drop2(x_1)
        x_1=self.bn3(x_1)
        x_1=self.lin3(x_1)
        return x_1

<p><img src='images/tip.png' width=720></p>

Tabular data uses shallow models with huge embedding tables and few feed-forward layers. 

In [9]:
# instantiate model
model=ProductPurchaseModel(embedding_sizes, len(cont_cols))
model.to(device)

ProductPurchaseModel(
  (embeddings): ModuleList(
    (0): Embedding(3303, 7)
    (1): Embedding(14, 3)
    (2): Embedding(61, 3)
    (3): Embedding(90, 3)
  )
  (emb_drop): Dropout(p=0.6, inplace=False)
  (bn1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin1): Linear(in_features=17, out_features=200, bias=True)
  (drop1): Dropout(p=0.3, inplace=False)
  (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop2): Dropout(p=0.3, inplace=False)
  (lin2): Linear(in_features=200, out_features=70, bias=True)
  (bn3): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin3): Linear(in_features=70, out_features=2, bias=True)
)

Next, we define a `torch.utils.data.Dataset` class to be use by `torch.utils.data.DataLoader`. The Dataset is makes it easier to track separate categorical and continuous variables. The DatalLoader wraps an iterable around the Dataset to enable easy access to the samples. More information about Dataset and DataLoader can be found in quick PyTorch [guide](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html). 

In [10]:
# define dataset
class myDataset(Dataset):
    def __init__(self, X, y, cat_cols, cont_cols):
        self.X_cat=torch.as_tensor(X.loc[:, cat_cols].copy().values.astype('int32'), device=device)
        self.X_cont=torch.as_tensor(X.loc[:, cont_cols].copy().values.astype('float32'), device=device)
        self.y=torch.as_tensor(y.astype('int64'), device=device)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx): 
        return self.X_cat[idx], self.X_cont[idx], self.y[idx]

In [11]:
# instantiate dataset
X_train=gdf[feature_cols]
y_train=gdf['target'].values

train_ds=myDataset(X_train, y_train, cat_cols, cont_cols)

<a name='s4-2.3'></a>
### Begin Training ###
We will set some parameters for training. 

In [12]:
# set optimizer
def get_optimizer(model, lr = 0.001, wd = 0.0):
    parameters=filter(lambda p: p.requires_grad, model.parameters())
    optim=torch_optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

In [13]:
# define training function
def train_model(model, optim, train_dl):
    # set the model to training, which is useful for BatchNorm and Dropout layers that behave differently during training and evaluation
    model.train()
    total=0
    sum_loss=0
    
    # iterate through batches
    for b, (X_cat, X_cont, y) in enumerate(train_dl):
        batch=y.shape[0]
        
        # forward pass
        output=model(X_cat, X_cont)
        
        # calculate loss
        loss=F.cross_entropy(output, y)
        
        # zero out the gradients so the parameters update correctly, otherwise gradients would be combined with old
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        # calculate total loss per batch
        total+=batch
        sum_loss+=batch*(loss.item())
    return sum_loss/total

Instantiate a `torch.utils.data.DataLoader` and begin training. 

In [14]:
# define training loop
def train_loop(model, epochs, lr=0.01, wd=0.0):
    # instantiate optimizer
    optim=get_optimizer(model, lr = lr, wd = wd)
    
    # iterate through number of epochs
    for i in tqdm(range(epochs)): 
        loss=train_model(model, optim, train_dl)
        print("training loss: ", round(loss, 3))

In [15]:
%%time

# define batch size and begin training
batch_size=1000
train_dl=DataLoader(train_ds, batch_size=batch_size, shuffle=True)

train_loop(model, epochs=3, lr=0.05, wd=0.00001)

 33%|███▎      | 1/3 [00:29<00:58, 29.50s/it]

training loss:  0.666


 67%|██████▋   | 2/3 [00:58<00:29, 29.27s/it]

training loss:  0.665


100%|██████████| 3/3 [01:27<00:00, 29.22s/it]

training loss:  0.665
CPU times: user 1min 27s, sys: 366 ms, total: 1min 28s
Wall time: 1min 27s


<a name='s4-3'></a>
## Visualizing the Embeddings ##

In [ ]:
# visualize embeddings

# import dependencies
import plotly.express as px
import pandas as pd

# pick category to visualize
category='brand'

category_label=pd.read_parquet(f'categories/unique.{category}.parquet')[category]
category_label=category_label[1:]

embeddings_idx=list(embedding_sizes.keys()).index(category)
embeddings=model.embeddings[embeddings_idx].weight.detach().cpu().numpy()

fig=px.scatter_3d(
    x=embeddings[:, 0], 
    y=embeddings[:, 1], 
    z=embeddings[:, 2], 
    text=category_label, 
    height=720
)
fig.show()

In [ ]:
# persist embeddings
!mkdir trained_embedding_weights

import pandas as pd
import numpy as np

for idx, each_col in enumerate(cat_cols): 
    weights=model.embeddings[idx].weight.detach().cpu().numpy()
    pd.DataFrame(weights).to_csv(f'trained_embedding_weights/{each_col}.csv', index=False)

<a name='s4-4'></a>
## Conclusion ##
Deep Learning is very good at feature extraction, which can be used for finding categorical embeddings. This is the advantage of using a Deep Learning approach, as it requires way less feature engineering and less dependent on domain knowledge. 

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>